# Laboratorio covid
## Cargar datos

In [10]:
from pyspark.sql.types import StringType,DoubleType,IntegerType, TimestampType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Load csv Dataset 
df=spark.read.csv('s3://ljpalaciomdatasets/Casos_positivos_de_COVID-19_en_Colombia.csv',inferSchema=True,header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: timestamp (nullable = true)
 |-- Codigo DIVIPOLA: integer (nullable = true)
 |-- Ciudad de ubicación: string (nullable = true)
 |-- Departamento o Distrito : string (nullable = true)
 |-- atención: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- País de procedencia: string (nullable = true)
 |-- FIS: string (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha diagnostico: timestamp (nullable = true)
 |-- Fecha recuperado: string (nullable = true)
 |-- fecha reporte web: timestamp (nullable = true)

## Borrar y crear algunas columnas
* Borré columnas **ID de caso, fecha reporte web, Codigo DIVIPOLA y Fecha de notificación**
* Por alguna razón extraña FIS (Fecha de inicio de Síntomas) estaba como un string, así que cambié el tipo a timestamp, ya que la necesitaba para crear una columna.
* Creé la columna **Horas entre primeros sintomas y diagnostico**
* Creé la columna **Tiene mas de 60**

In [180]:
#drop column of dataframe
df_new=df.drop('ID de caso', 'fecha reporte web', 'Codigo DIVIPOLA', 'Fecha de notificación')
df_new.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Ciudad de ubicación', 'Departamento o Distrito ', 'atención', 'Edad', 'Sexo', 'Tipo', 'Estado', 'País de procedencia', 'FIS', 'Fecha de muerte', 'Fecha diagnostico', 'Fecha recuperado']

In [73]:
#cast column
df_new = df_new.withColumn("FIS",(df_new['FIS'].cast(TimestampType())))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [181]:
# New column, Horas entre primeros sintomas y diagnostico
df_new = df_new.withColumn(
    "Horas entre primeros sintomas y diagnostico", 
    (df_new["Fecha diagnostico"].cast("long") - df_new["FIS"].cast("long"))/3600.
)
df_new.select("Fecha diagnostico", "FIS", "Horas entre primeros sintomas y diagnostico").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+-------------------------------------------+
|  Fecha diagnostico|                 FIS|Horas entre primeros sintomas y diagnostico|
+-------------------+--------------------+-------------------------------------------+
|2020-03-06 00:00:00|2020-02-27T00:00:...|                                       null|
|2020-03-09 00:00:00|2020-03-04T00:00:...|                                       null|
|2020-03-09 00:00:00|2020-02-29T00:00:...|                                       null|
|2020-03-11 00:00:00|2020-03-06T00:00:...|                                       null|
|2020-03-11 00:00:00|2020-03-08T00:00:...|                                       null|
+-------------------+--------------------+-------------------------------------------+
only showing top 5 rows

In [183]:
# New column tiene mas de 60
df_new =df_new.withColumn("Tiene mas de 60", df_new["Edad"] > 60)
df_new.select("Edad", "Tiene mas de 60").show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+---------------+
|Edad|Tiene mas de 60|
+----+---------------+
|  19|          false|
|  34|          false|
|  50|          false|
|  55|          false|
|  25|          false|
|  27|          false|
|  85|           true|
|  22|          false|
|  28|          false|
|  36|          false|
+----+---------------+
only showing top 10 rows

## Realizar filtrados de datos por alguna información que le parezca interesante

* El primer filtro consiste en ver los menores de edad que están en grave estado de salud o que han fallecido. Se observa que lamentablemente hay bebés que han fallecido y los otros menores están en cuidados intensivos.
* El segundo filtro consiste en ver los casos del virus procedentes de otros países. Se puede observar que son bastantes

In [184]:
#Distinct Values in a column
df_new.select('atención').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+
|    atención|
+------------+
|        CASA|
|Hospital UCI|
|   Fallecido|
|        Casa|
|    Hospital|
|         N/A|
|        casa|
|  Recuperado|
+------------+

In [100]:
#Distinct Values in a column
df_new.select('estado').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+
|   estado|
+---------+
|Fallecido|
| Moderado|
|      N/A|
|     leve|
|     Leve|
|    Grave|
+---------+

In [188]:
filter1 = df_new.filter((df_new["estado"] == "Grave") | (df_new["estado"] == "Fallecido")).filter(
    df_new["Edad"] <= 18
).select(
    "Edad", "atención", "estado",  "Ciudad de ubicación", "Sexo", "Fecha diagnostico", "País de procedencia"
).orderBy("Edad")
filter1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------------+---------+-------------------+----+-------------------+-------------------+
|Edad|    atención|   estado|Ciudad de ubicación|Sexo|  Fecha diagnostico|País de procedencia|
+----+------------+---------+-------------------+----+-------------------+-------------------+
|   1|   Fallecido|Fallecido|             Samacá|   F|2020-05-01 00:00:00|           Colombia|
|   1|   Fallecido|Fallecido|       Cundinamarca|   M|2020-04-20 00:00:00|           Colombia|
|   3|   Fallecido|Fallecido|            Popayán|   F|2020-04-06 00:00:00|           Colombia|
|  12|Hospital UCI|    Grave|        Bogotá D.C.|   F|2020-05-01 00:00:00|           Colombia|
|  13|Hospital UCI|    Grave|        Bogotá D.C.|   M|2020-04-24 00:00:00|           Colombia|
|  18|Hospital UCI|    Grave|               Cali|   F|2020-04-19 00:00:00|           Colombia|
+----+------------+---------+-------------------+----+-------------------+-------------------+

In [189]:
filter2 = df_new.filter(
   (df_new["Ciudad de ubicación"] == "Medellín") & (df_new["Tipo"] == "Importado")
).select(
    "Edad", "atención", "estado",  "Ciudad de ubicación", "Sexo", "Fecha diagnostico", "País de procedencia"
).orderBy("Edad")
filter2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------------+------+-------------------+----+-------------------+--------------------+
|Edad|    atención|estado|Ciudad de ubicación|Sexo|  Fecha diagnostico| País de procedencia|
+----+------------+------+-------------------+----+-------------------+--------------------+
|  15|  Recuperado|  Leve|           Medellín|   F|2020-04-07 00:00:00|Estados Unidos de...|
|  21|  Recuperado|  Leve|           Medellín|   F|2020-03-23 00:00:00|              España|
|  21|  Recuperado|  Leve|           Medellín|   M|2020-03-23 00:00:00|Estados Unidos de...|
|  21|  Recuperado|  Leve|           Medellín|   F|2020-03-23 00:00:00|              España|
|  22|  Recuperado|  Leve|           Medellín|   F|2020-03-20 00:00:00|              España|
|  23|        Casa|  Leve|           Medellín|   F|2020-04-03 00:00:00|             Turquía|
|  24|        Casa|  Leve|           Medellín|   F|2020-03-28 00:00:00|Estados Unidos de...|
|  24|  Recuperado|  Leve|           Medellín|   F|2020-03-29 00:00:00

# Realizar alguna agrupación y consulta de datos categorica, por ejemplo número de casos por región o por sexo/genero.
## Mostré el número de casos graves por Departamento 

In [190]:
# Value counts
group = df_new.filter(df_new["estado"] == "Grave").select(
    "Edad", "atención", "estado",  "Ciudad de ubicación", "Sexo", "Fecha diagnostico", "País de procedencia", 'Departamento o Distrito '
).groupBy('Departamento o Distrito ', "estado").count().orderBy('count', ascending=False)
group.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+------+-----+
|Departamento o Distrito |estado|count|
+------------------------+------+-----+
|             Bogotá D.C.| Grave|   39|
|         Valle del Cauca| Grave|   36|
|               Antioquia| Grave|   12|
|     Cartagena D.T. y C.| Grave|    6|
|               Atlántico| Grave|    4|
|                  Nariño| Grave|    4|
|                   Huila| Grave|    3|
|       Barranquilla D.E.| Grave|    3|
|               Risaralda| Grave|    2|
|                   Cesar| Grave|    2|
|       Buenaventura D.E.| Grave|    2|
|                  Tolima| Grave|    1|
|                    Meta| Grave|    1|
|    Santa Marta D.T. ...| Grave|    1|
|                 Bolívar| Grave|    1|
|              La Guajira| Grave|    1|
|      Norte de Santander| Grave|    1|
|            Cundinamarca| Grave|    1|
+------------------------+------+-----+

In [193]:
#save the dataframe as single csv
base_url = "s3://ljpalaciomdatasets/covid"
# write data
df_new.coalesce(1).write.format("csv").option("header","true").save(base_url + "/df_new")
filter1.coalesce(1).write.format("csv").option("header","true").save(base_url + "/filter1")
filter2.coalesce(1).write.format("csv").option("header","true").save(base_url + "/filter2")
group.coalesce(1).write.format("csv").option("header","true").save(base_url + "/group")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…